In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
test_csv = pd.read_csv("/opt/ml/dataset/test/test_data.csv")
submission_csv = pd.read_csv("./prediction/submission.csv")
sota_csv = pd.read_csv("./prediction/submission_jh.csv")

## 1. "예측 / SOTA 값" 비교 (태일)

In [3]:
sentences = test_csv["sentence"]
subject_entities, object_entities = test_csv["subject_entity"], test_csv["object_entity"] 
my_labels = submission_csv["pred_label"]
sota_labels = sota_csv["pred_label"]

In [4]:
i = -1
cnt = 0
# while i < 50: # 전체 텍스트: len(my_labels) - 1

while i < len(my_labels)-1:
    i += 1
    if my_labels[i] == sota_labels[i]:
        continue
    cnt += 1
    print(f"{cnt}: {sentences[i]}")
    print(f"sub:{eval(subject_entities[i])['word']}, obj:{eval(object_entities[i])['word']}")
    print(f"예측: {my_labels[i]}, sota: {sota_labels[i]}")
    print()
print(f"num of diff: {cnt} / {i}")

1: 당시 민주당 이진련 시의원은 권영진 시장에게 "긴급 생계자금을 왜 현금으로 지원하지 않느냐"고 따졌다.
sub:민주당, obj:권영진
예측: org:top_members/employees, sota: no_relation

2: 소련군에서 그의 이름도 중국식의 진지첸으로 조선인으로서의 정체성도 희박했다.
sub:소련군, obj:진지첸
예측: no_relation, sota: org:top_members/employees

3: 한편, 시는 3E 정책에 대한 아이디어가 있는 시민들은 누구나 정책을 제안 할 수 있도록 시청 홈페이지 ‘정책소통 1번가’를 통해 온-라인으로도 제안을 접수받고 있다.
sub:시, obj:3E
예측: no_relation, sota: org:product

4: 동시에 울라슬로 2세는 게오르크와 결혼한 베아트릭스를 통해 크로아티아 지역의 반발 및 국경분쟁에 대처하려 했다.
sub:울라슬로 2세, obj:게오르크
예측: per:spouse, sota: no_relation

5: 이날 협약식에는 경남대 최호성 대외부총장과 거제 라마다 스위츠 거제호텔 정행철 대표이사를 비롯해 경남대 관광학부 선종갑, 박한수 교수 및 관계자가 참석한 가운데 진행됐다.
sub:라마다 스위츠 거제호텔, obj:경남대 관광학부
예측: no_relation, sota: org:member_of

6: 올해 동신대 수시 모집의 가장 큰 특징은 지역인재전형의 비중을 크게 늘리며 광주와 전남·북 지역 학생들의 입학 기회를 넓혔다는 점이다.
sub:동신대, obj:전남
예측: org:place_of_headquarters, sota: no_relation

7: 김승남 전 위원장은 “우리 정치의 가장 시급한 과제는 고통 받는 농어촌과 서민들이 다시 일어서서 행복하게 삶을 영위할 수 있도록 정책을 발굴하고 이를 뒷받침할 수 있는 법안과 예산을 만드는 일이며, 이를 위해서는 혁신적인 농어업지원정책이 마련되어야 한다”고 주장했다.
sub:김승남, obj:위원장
예측: pe

## csv파일로 뽑기(나연 추가)

In [5]:
from load_data import *
from train import *

test_csv = pd.read_csv("/opt/ml/dataset/test/test_data.csv")
test_csv = preprocessing_dataset(test_csv, marking_mode='normal')

res_df = pd.DataFrame()
res_df['sentence'] = test_csv['sentence']
res_df['sub'] = test_csv["subject_entity"]
res_df['obj'] = test_csv["object_entity"] 

res_df['sota'] = sota_csv["pred_label"]
res_df['my'] = submission_csv["pred_label"]

diff_df = res_df[res_df['sota'] != res_df['my']]  # sota와 다른 부분만
diff_df.to_csv("sub_eda.csv", index=False, encoding="utf-8-sig")  # 저장
diff_df
# res_df

/opt/conda/envs/klue/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,sentence,sub,obj,sota,my
4,"당시 민주당 이진련 시의원은 권영진 시장에게 ""긴급 생계자금을 왜 현금으로 지원하지...",민주당,권영진,no_relation,org:top_members/employees
9,소련군에서 그의 이름도 중국식의 진지첸으로 조선인으로서의 정체성도 희박했다.,소련군,진지첸,org:top_members/employees,no_relation
12,"한편, 시는 3E 정책에 대한 아이디어가 있는 시민들은 누구나 정책을 제안 할 수 ...",시,3E,org:product,no_relation
20,동시에 울라슬로 2세는 게오르크와 결혼한 베아트릭스를 통해 크로아티아 지역의 반발 ...,울라슬로 2세,게오르크,no_relation,per:spouse
24,이날 협약식에는 경남대 최호성 대외부총장과 거제 라마다 스위츠 거제호텔 정행철 대표...,라마다 스위츠 거제호텔,경남대 관광학부,org:member_of,no_relation
...,...,...,...,...,...
7748,그러자 다윗은 요압 대신 요압의 어머니 스루야의 자매 나하스의 딸 아비가일과 혼인 ...,나하스,다윗,per:siblings,no_relation
7750,그룹 트와이스 멤버 정연(유정연·23)이 멤버 지효 부상 소식에 팬들에게 당부의 말...,트와이스,정연,per:employee_of,org:top_members/employees
7753,미국군은 마지막 추축국인 일본의 본토에 무차별 야간 공습을 실시했다.,미국군,일본,no_relation,org:place_of_headquarters
7754,황병직(영주) 의원은 대구경북연구원의 규정을 위반한 직원평정과 이로 인한 성과급 과...,황병직,영주,per:place_of_residence,per:employee_of


## 2. Check for Submission Distribution (찬국)
현재 자신의 모델이 잘 예측되었는지 시각적으로 확인하기 위한 코드입니다.  
해당 코드는 'no-relation으로 예측한 결과가 적으면, 그만큼 다른 label을 예측할 수 있기에 좋은 성능을 기대할 수 있을 것이다'라는 가설을 바탕으로 작성되었습니다.  
비교를 위해 리더보드에서 f1-score 70점을 달성한 재학님의 모델 예측 결과와 현재 자신의 submission의 분포를 비교할 수 있습니다.  
- 해당 코드는 distribution 확인을 위한 코드입니다. 기준 예측 결과는 추후 개선할 수 있습니다. 

In [ ]:
# first_dataset
label_jh = sota_csv.groupby('pred_label').count().sort_values(by = 'id', ascending=False)
label_jh.reset_index(level=['pred_label'], inplace = True)
label_jh.drop('probs', axis=1, inplace=True)
label_jh.columns = ["pred_label", "count"]

In [ ]:
# second_dataset
label = submission_csv.groupby('pred_label').count().sort_values(by = 'id', ascending=False)
label.reset_index(level=['pred_label'], inplace = True)
label.drop('probs', axis=1, inplace=True)
label.columns = ["pred_label", "count"]

## 2-1.Distribution

In [ ]:
# 병렬적으로 확인하고 싶다면, subplot을 1,2로 주고 0ax 부분을 주석해제시키면 됩니다. 
fig, axes = plt.subplots(1, 1, figsize=(10, 8))
sns.barplot(y='pred_label', x='count', data=label_jh, # ax = axes[0],
              color='red',alpha=0.5 ) # pallete:

sns.barplot(y='pred_label', x='count', data=label, # ax = axes[1]
              color='gray',alpha=0.5 )
plt.title("Submission Distribution")

## 2-2.Difference

In [ ]:
diff = label_jh['count'] - label['count']
diff_df =pd.DataFrame({"pred_label": label_jh['pred_label'], "diff":diff})

In [ ]:
# 병렬적으로 확인하고 싶다면, subplot을 1,2로 주고 0ax 부분을 주석해제시키면 됩니다. 
fig, ax = plt.subplots(figsize=(12, 10))
sns.barplot(y='pred_label', x='diff', data=diff_df, # ax = axes[0],
              palette='flare')

for i in ax.containers:
    ax.bar_label(i, fontsize=13)